# Notebook 4: Compute d3 for testing facilities and residence centroids and visualize routes

## Introduction to Noteboook 4

In this notebook, we will:

1. compute `d3` using several methods
2. Obtain `d3_total` by adding `d1`, `d2` and `d3` from the selected method
3. Display residence centroids, testing facilities, a sample of 5 routes.

### Note

This notebook loads two graphs (projected and unprojected) and hence would consume a lot more memory. You should shut down other notebooks if you have memory constraints.

## Data sources

In [1]:
import pandas as pd, geopandas as gpd, folium

pd.options.display.float_format = '{:.10f}'.format

### Testing Facilities, Target Parish, Parishes, Residence Centroids

In [2]:
filtered_testing_sites_4326_gdf = gpd.read_file('data/filtered_testing_sites_4326_gdf.gpkg')
parish_gdf = gpd.read_file('data/parish_gdf.gpkg')
parishes_gdf = gpd.read_file('data/parishes_gdf.gpkg')
residential_centroids_4326_gdf = gpd.read_file('data/residential_centroids_4326_gdf.gpkg')

In [3]:
residential_centroids_4326_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   parish_name  214 non-null    object  
 1   building     214 non-null    object  
 2   lat          214 non-null    float64 
 3   lon          214 non-null    float64 
 4   prj_lat      214 non-null    float64 
 5   prj_lon      214 non-null    float64 
 6   r_node       214 non-null    int64   
 7   r_node_lat   214 non-null    float64 
 8   r_node_lon   214 non-null    float64 
 9   r_node_y     214 non-null    float64 
 10  r_node_x     214 non-null    float64 
 11  d1           214 non-null    float64 
 12  d1_euc       214 non-null    float64 
 13  geometry     214 non-null    geometry
dtypes: float64(10), geometry(1), int64(1), object(2)
memory usage: 23.5+ KB


In [4]:
residential_centroids_4326_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
residential_centroids_4326_gdf

,parish_name,building,lat,lon,prj_lat,prj_lon,r_node,r_node_lat,r_node_lon,r_node_y,r_node_x,d1,d1_euc,geometry
0,Kisugu,residential,0.3050501833,32.6082495446,33718.0074585697,456408.2543575270,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,50.2749796310,50.2749796310,POINT (32.60825 0.30505)
1,Kisugu,dormitory,0.3053414841,32.6082456333,33750.2057395246,456407.8202964029,579993314,0.3055411000,32.6084585000,33772.2689535378,456431.5080539904,32.3712105445,32.3712105445,POINT (32.60825 0.30534)
2,Kisugu,duplex,0.3045664193,32.6084960943,33664.5347349995,456435.6874821403,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,21.9934580881,21.9934580881,POINT (32.60850 0.30457)
3,Kisugu,residential,0.3045408000,32.6083606500,33661.7035135770,456420.6156729474,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,9.1897739823,9.1897739823,POINT (32.60836 0.30454)
4,Kisugu,residential,0.3046784338,32.6080187078,33676.9179346887,456382.5662585886,6227266299,0.3045855000,32.6082708000,33666.6446875843,456410.6177021883,29.8734513255,29.8734513255,POINT (32.60802 0.30468)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Kisugu,house,0.3086906479,32.6082211451,34120.3978818522,456405.1089029149,579993370,0.3084251000,32.6083627000,34091.0455836289,456420.8594871131,33.3112340446,33.3112340446,POINT (32.60822 0.30869)
210,Kisugu,house,0.3088099936,32.6081564965,34133.5897534508,456397.9155612630,1460122475,0.3091566000,32.6080193000,34171.9016473888,456382.6502965369,41.2411144888,41.2411144888,POINT (32.60816 0.30881)
211,Kisugu,house,0.3087290884,32.6080501563,34124.6475147437,456386.0821231636,579993372,0.3083446000,32.6082158000,34082.1483045059,456404.5127182511,46.3235329624,46.3235329624,POINT (32.60805 0.30873)
212,Kisugu,house,0.3090996240,32.6081822843,34165.6032674131,456400.7863013543,6226990677,0.3092163000,32.6081950000,34178.4997268183,456402.2017287260,12.9739007177,12.9739007177,POINT (32.60818 0.30910)


In [6]:
filtered_testing_sites_4326_gdf.shape

(16, 17)

In [7]:
filtered_testing_sites_4326_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   FACILITY     16 non-null     object  
 1   CITY         16 non-null     object  
 2   ADDRESS      16 non-null     object  
 3   LAT          16 non-null     float64 
 4   LON          16 non-null     float64 
 5   COORDINATES  16 non-null     object  
 6   NOTES        2 non-null      object  
 7   PRJ_LAT      16 non-null     float64 
 8   PRJ_LON      16 non-null     float64 
 9   t_node       16 non-null     int64   
 10  t_node_lat   16 non-null     float64 
 11  t_node_lon   16 non-null     float64 
 12  t_node_y     16 non-null     float64 
 13  t_node_x     16 non-null     float64 
 14  d2           16 non-null     float64 
 15  d2_euc       16 non-null     float64 
 16  geometry     16 non-null     geometry
dtypes: float64(10), geometry(1), int64(1), object(5)
memory usage: 2.2+

In [8]:
filtered_testing_sites_4326_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
filtered_testing_sites_4326_gdf

,FACILITY,CITY,ADDRESS,LAT,LON,COORDINATES,NOTES,PRJ_LAT,PRJ_LON,t_node,t_node_lat,t_node_lon,t_node_y,t_node_x,d2,d2_euc,geometry
0,Central Public Health Laboratory,Kampala,"7/11, Plot 113 Buganda Rd, Kampala, Uganda",0.3312460000,32.5761710000,"0.331245631028126, 32.57617147103373",None,36613.6482006860,452838.7936172243,3799704477,0.3311248000,32.5760535000,36600.2521446763,452825.7180929286,18.7196060915,18.7196060915,POINT (32.57617 0.33125)
1,Infectious Disease Institute Laboratory,Kampala,"P.O.Box 22418, Kampala, Uganda",0.3391550000,32.5761190000,"0.3391550027171229, 32.57611913788221",None,37487.8549663987,452833.0450933628,7401202859,0.3390506000,32.5759309000,37476.3162241753,452812.1135546908,23.9012945941,23.9012945941,POINT (32.57612 0.33916)
2,Makerere University,Kampala,"University Rd, Kampala, Uganda",0.3337660000,32.5675150000,"0.33376643025242, 32.56751532874441",None,36892.2336127046,451875.5986544515,2297820937,0.3338064000,32.5674293000,36896.6995755768,451866.0624737106,10.5301266611,10.5301266611,POINT (32.56752 0.33377)
3,Mild May Laboratory,Kampala,"6HG2+QJH, Kampala, Uganda",0.2272610000,32.5514940000,"0.22726149143899727, 32.551493611083714",None,25119.9449333336,450092.3882960156,2614743709,0.2274452000,32.5514859000,25140.3052270359,450091.4875805033,20.3802072629,20.3802072629,POINT (32.55149 0.22726)
4,Joint Clinical Research Center (JCRC),Kampala,"P.o.Box 10005, Kampala, Uganda",0.2471060000,32.5615450000,"0.24710642516379605, 32.56154518525522",None,27313.4443953668,451210.9074285642,7062105534,0.2489300000,32.5610400000,27515.0590496562,451154.7191193702,209.2978617050,209.2978617050,POINT (32.56155 0.24711)
5,MBN Laboroatory,Kampala,"Plot 28 Nakasero Rd, Kampala, Uganda",0.3244010000,32.5768040000,"0.3244006304886406, 32.57680365762819",None,35857.0460002950,452909.1994618282,6880975575,0.3244436000,32.5767374000,35861.7550210679,452901.7886607080,8.7803672978,8.7803672978,POINT (32.57680 0.32440)
6,Medipal International Hospital,Kampala,"John Babiha (Acacia) Ave, Kampala, Uganda",0.3267710000,32.5876990000,"0.32677070175063294, 32.58769862875094",None,36118.9590743176,454121.5640353452,8193448456,0.3265384000,32.5878109000,36093.2485964963,454134.0147725081,28.5665805742,28.5665805742,POINT (32.58770 0.32677)
7,Test and Fly Laboratory,Kampala,"Yusuf Lule Road, Kampala, Uganda",0.3280000000,32.5833240000,"0.3279995303971809, 32.58332419987116",None,36254.8242478438,453634.7367577547,7238684605,0.3280009000,32.5832163000,36254.9242265742,453622.7523243336,11.9848504443,11.9848504443,POINT (32.58332 0.32800)
8,Uganda Cancer Institute,Kampala,"Upper Mulago Hill Rd, Kampala, Uganda",0.3415660000,32.5779390000,"0.34156560138915515, 32.577938699857306",None,37754.3414306474,453035.5792187499,6232768975,0.3417020000,32.5779317000,37769.3739685455,453034.7675639616,15.0544338702,15.0544338702,POINT (32.57794 0.34157)
9,IOM Laboratory,Kampala,"Plot 6A Bukoto Crescent, Naguru, Kampala 11431...",0.3419140000,32.6050250000,"0.3419138946476636, 32.6050250521126",None,37792.6787080598,456049.5983998349,560476404,0.3423478000,32.6052258000,37840.6268860078,456071.9445241932,52.8996884903,52.8996884903,POINT (32.60502 0.34191)


### Projectted OSMNx graph for Kampala

In [10]:
import osmnx as ox, csv

with open('overpass-api.csv', mode='r') as infile:
    reader = csv.reader(infile)
    overpass_api = {rows[0]:rows[1] for rows in reader}

ox.config(
    log_console=False, 
    use_cache=True, 
    log_file=True,
    overpass_endpoint=overpass_api['main']
)

In [11]:
%%time
if 'G_proj' not in globals():
    G_proj = ox.load_graphml('data/g_projected.graphml')

if 'G' not in globals():
    G = ox.load_graphml('data/g_unprojected.graphml')

CPU times: user 37.8 s, sys: 906 ms, total: 38.7 s
Wall time: 38.7 s


In [12]:
G_proj.graph['crs']

'+proj=utm +zone=36 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs'

In [13]:
G.graph['crs']

'epsg:4326'

## Create residence-test facility pair DF and analysis columns

### D3 computation approaches

1. Method 1 - Sum of euclidean distances between nodes that constitute a path (d3_path_sum)
2. Method 2 - Total length of a Shapely LineString derived from list of coordinates that constitute a path (d3_shapely)
3. Method 3 - Simple euclidean distance between closest node to a residence centroid and the closest node to a testing facility (d3_euc)
4. Method 4 - Computed through OSMNx graph utils' route_edge_attributes function using "length" as weight

All methods, except Method 3, use the list of path nodes (path_node_list) generated from OSMNx shortest path function.

All methods, except Method 4, use a projected graph. 

References:

1. Method 2: https://shapely.readthedocs.io/en/stable/manual.html
2. Method 3: https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.euclidean_dist_vec
3. Method 4: https://github.com/gboeing/osmnx-examples/blob/main/notebooks/02-routing-speed-time.ipynb

### Create a Pandas DataFrame (DF) to store results of d1, d2, and d3 computation

In [14]:
%%time
import networkx as nx, os
from tqdm import tqdm, notebook
from shapely.geometry import LineString


dict_list = []
df_list = []

# tqdm parameters
total_rows=5 #residential_centroids_4326_gdf.shape[0]

for r_index, r_row in tqdm(residential_centroids_4326_gdf.sample(n=total_rows, random_state=1).iterrows(), total=total_rows, desc='Residence Loop'):
    
    # for each sampled residence centroid, compute d3 and other parameters
    # create a dictionary for each pair and create a temporary DF for each dictionary
    dict_list = []
    for t_index, t_row in filtered_testing_sites_4326_gdf.iterrows():
        
        # 1. assemble the paired record using the nested loop
        r_parish_name = r_row['parish_name']
        r_node = r_row['r_node']
        r_node_lat = r_row['r_node_lat']
        r_node_lon = r_row['r_node_lon']
        t_node = t_row['t_node']
        t_node_lat = t_row['t_node_lat']
        t_node_lon = t_row['t_node_lon']
        d1 = r_row['d1']
        d2 = t_row['d2']
        
        # List if OSMNx nodes for shortest path
        path_node_list = ox.distance.shortest_path(G_proj, r_node, t_node, weight='length', cpus=2)

        ## d3 method 1 (sum of euclidean distance of path edges)
        # simplified explanation: 
        # 1. node to node distance: osmnx.distance.euclidean_dist_vec(source_lat, source_lon, target_lat, target_lon, distance)
        # 2. dist_list: list of node to node distances generated through Python list comprehension (for loop)
        # 3. d3_path_sum: sum of distances in dist_list
        dist_list = [ ox.distance.euclidean_dist_vec(G_proj.nodes[path_node_list[i]]['y'], G_proj.nodes[path_node_list[i]]['x'], \
                                                  G_proj.nodes[path_node_list[i+1]]['y'], G_proj.nodes[path_node_list[i+1]]['x']) for i in range(len(path_node_list)-1) ]
        d3_path_sum = sum(dist_list)

        ## d3 method 2 (Shapely LineString Length)
        coords_list = [(G_proj.nodes[node]['x'], G_proj.nodes[node]['y']) for node in path_node_list ]
        path_line = LineString(coords_list)
        d3_shapely = path_line.length

        ## d3 method 3 (euclidean distance between residence centroid and test facility coordinates
        d3_euc = ox.distance.euclidean_dist_vec(r_row['prj_lat'], r_row['prj_lon'], t_row['PRJ_LAT'], t_row['PRJ_LON'])
        
        ## d3 method 4 (sum of lengths of edges through OSMNx edge attributes
        d3_edge_attrs = sum(ox.utils_graph.get_route_edge_attributes(G, path_node_list, "length"))
        
        # obtain d_total
        d_total = d1 + d2 + d3_edge_attrs
        
        # assemble record dictionary
        record_dict = {
            'parish_name': r_parish_name,
            'r_node': r_node,
            'r_node_lat': r_node_lat,
            'r_node_lon': r_node_lon,
            't_node': t_node,
            't_node_lat': t_node_lat,
            't_node_lon': t_node_lon,
            'd3_euc': d3_euc,
            'd3_shapely': d3_shapely,
            'path_node_list': path_node_list,
            'path_distances': dist_list,
            'd3_path_sum': d3_path_sum,
            'd3_edge_attrs': d3_edge_attrs,
            'd_total': d_total
        }
        #print(record_dict)
        
        # append record dict to dict_list (each record is a future DF row, each dict_list is future temp DF)
        dict_list.append(record_dict)
    
    # 2. Create a temporary DF to store list of record dictionaries (these become DF rows)
    # Sort values by the d3_path_sum column
    # Group by r_node (remember each temp DF contains records for only one r_node)
    # Retain the first record (which would have the smallest value of d_total)
    df = pd.DataFrame(dict_list).sort_values(by=['d_total']).groupby(['r_node'], as_index=False).first()
    # append this temp DF to a DF list
    df_list.append(df)

# 3. Concatenate (combine) all the temp DFs in the DF list to a single DF
# Reset the index of the DF and drop the original indices of the temp DFs
paired_df = pd.concat(df_list).reset_index(drop=True)

# 4. Convert the path_node_list column to type string
paired_df['path_node_list'] = paired_df['path_node_list'].astype(str)

paired_df

Residence Loop: 100%|██████████| 5/5 [00:36<00:00,  7.21s/it]

CPU times: user 35.5 s, sys: 657 ms, total: 36.1 s
Wall time: 36.1 s


,r_node,parish_name,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d3_edge_attrs,d_total
0,475033488,Kisugu,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2622.6930000000,2729.7292195052
1,579993375,Kisugu,0.3082407000,32.6077652000,7401065633,0.3076673000,32.6249390000,1864.5203716102,3031.4086806217,"[579993375, 579993374, 579993373, 6226994993, ...","[19.542746737921256, 16.795537250722806, 8.728...",3031.4086806217,3037.0040000000,3151.0671117949
2,579993404,Kisugu,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2730.6216586954,"[579993404, 1583799967, 1583799788, 1583799935...","[32.71856677381537, 26.624433957601738, 25.730...",2730.6216586954,2735.6840000000,2840.9248157841
3,1583799787,Kisugu,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2682.0780000000,2782.0532660583
4,8144996943,Kisugu,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2776.6460000000,2802.9625482335


## Map Paired Residence Centroids and Test Facilities and Node-to-Node Routes

In [15]:
%%time
import folium, json
from folium import plugins

map1 = filtered_testing_sites_4326_gdf.explore(marker_kwds=dict(radius=5))

# Tile Layer (can add more, these become radio buttons on Layer Control)
folium.TileLayer('cartodbpositron').add_to(map1)
folium.TileLayer('cartodbdark_matter').add_to(map1)

# Feature groups become checkboxes in Layer Control Widget
fg1=folium.FeatureGroup(name='Residences', show=True)
fg2=folium.FeatureGroup(name='Residence Nodes', show=True)
fg3=folium.FeatureGroup(name='Residence to Node', show=True)

# Residences, Nodes and Residence-to-Node Paths
for index,row in residential_centroids_4326_gdf.iterrows():

    folium.CircleMarker(
                    location=[row['lat'],row['lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='red', \
                    fill_opacity=1).add_to(fg1)


    folium.CircleMarker(
                    location=[row['r_node_lat'],row['r_node_lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='yellow', \
                    fill_opacity=1).add_to(fg2)
    
    r_line_points = ((row['lat'],row['lon']),(row['r_node_lat'],row['r_node_lon']))
    popup_d1 = folium.Popup(str(row['d1'])+' meters')
    folium.PolyLine(r_line_points,
                    color='gray',
                    popup=popup_d1,
                    weight=1,
                    opacity=0.8
                   ).add_to(fg3)

fg1.add_to(map1)
fg2.add_to(map1)
fg3.add_to(map1)

# Residences, Nodes and Residence-to-Node Paths
fg4=folium.FeatureGroup(name='Testing Sites', show=True)
fg5=folium.FeatureGroup(name='Testing Site Nodes', show=True)
fg6=folium.FeatureGroup(name='Testing Site to Node', show=True)

for index,row in filtered_testing_sites_4326_gdf.iterrows():

    folium.CircleMarker(
                    location=[row['LAT'],row['LON']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='blue', \
                    fill_opacity=1).add_to(fg4)


    folium.CircleMarker(
                    location=[row['t_node_lat'],row['t_node_lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='yellow', \
                    fill_opacity=1).add_to(fg5)
    
    t_line_points = ((row['LAT'],row['LON']),(row['t_node_lat'],row['t_node_lon']))
    popup_d2 = folium.Popup(str(row['d2'])+' meters')
    folium.PolyLine(t_line_points,
                    color='gray',
                    popup=popup_d2,
                    weight=1,
                    opacity=0.8
                   ).add_to(fg6)
fg4.add_to(map1)
fg5.add_to(map1)
fg6.add_to(map1)

# Node to Node Routes
fg7=folium.FeatureGroup(name='Node to Node Routes', show=True)
for index, row in paired_df.sample(n=5,random_state=1).iterrows():
    path_string = row['path_node_list']
    path_node_list = json.loads(path_string)
    edge_list = []
    for i in range(len(path_node_list)-1):
        edge_pair = ((G_proj.nodes[path_node_list[i]]['lat'], G_proj.nodes[path_node_list[i]]['lon']), \
                     (G_proj.nodes[path_node_list[i+1]]['lat'], G_proj.nodes[path_node_list[i+1]]['lon']))
        edge_list.append(edge_pair)
        popup = folium.Popup(str(row['d_total']/1000)+' km')
        folium.PolyLine(edge_list,
                        color='red',
                        weight=2,
                        opacity=0.8,
                        popup=popup
                       ).add_to(fg7)
fg7.add_to(map1)

# GeoJSON parish boundary with style function
fg8=folium.FeatureGroup(name='Parish Boundary', show=True)
parish_name = ''.join(paired_df['parish_name'].unique())
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0, 
                            'weight': 3}
geojson = folium.GeoJson(
    data=parish_gdf['geometry'], 
    name="geojson",
    style_function=style_function
)
geojson.add_child(folium.Popup('Parish Boundary: '+parish_name))
geojson.add_to(fg8)
fg8.add_to(map1)

# Let's give the user the option to turn this feature group on (set it to False).
#  This feature group overlaps with feature group 8.
fg9=folium.FeatureGroup(name='Parishes', show=False)
for _, r in parishes_gdf.iterrows():
    # Simplify the representation of the parishes
    #    so the polygons display easily
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.00001)
    # convert the simplified geometry to GeoJSON
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'color':'black','weight':2, 'fillColor': 'orange','fillOpacity':0.05},
                           highlight_function=lambda x: {'color':'black','weight':2, 'fillColor': 'blue','fillOpacity':0.075})
    # prepare the popup for each parish (name and area in sq m)
    folium.Popup('Parish: '+r['NAME_4']+'<br/>'+'Area (sq. m.): '+str(r['area_sqm'])).add_to(geo_j)
    geo_j.add_to(fg9)
fg9.add_to(map1)

# The following are map controls:
# 1. Layer Control
folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map1)
# 2. Measure Control
map1.add_child(plugins.MeasureControl(activecolor = "blue", completedcolor = "black",))
# 3. Full Screen button
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map1)

map1

CPU times: user 502 ms, sys: 36.4 ms, total: 539 ms
Wall time: 608 ms


## Housekeeping

In [16]:
paired_df.to_pickle('data/paired_df.pickle')

In [17]:
if os.path.exists('data/paired_cache_df.pickle'):
    paired_cache_df = pd.read_pickle('data/paired_cache_df.pickle')
else:
    paired_cache_df = pd.DataFrame()

if (paired_cache_df.empty) or (parish_name not in paired_cache_df['parish_name'].values):
    paired_cache_df = paired_cache_df.append(paired_df, ignore_index=True)
    paired_cache_df.to_pickle('data/paired_cache_df.pickle')

paired_cache_df['parish_name'].unique()

array(['Kabowa', 'Kisugu', 'Kawempe I', 'Bogolobi'], dtype=object)